Data path and imports

In [27]:
import pandas as pd
from scipy.stats import chisquare
from scipy.stats import chi2_contingency
DATA_PATH = "data/BazaIzobrazevanje.xls"

Reading XLS file and getting appropriate columns

In [28]:
df = pd.read_excel(DATA_PATH)
cols = []
for col in df.columns[11:]:
    cols.append(col)

Get Chi Square

In [29]:
chi_square_list = []
for col in cols:
    contigency = pd.crosstab(df["Spol"], df[col])
    c, p, dof, expected = chi2_contingency(contigency)
    chi_square_list.append((c,p))
print(chi_square_list)

[(1.6124188466809726, 0.9999999846035881), (15.631866441667107, 0.11066677443489403), (30.691552715362235, 0.0006596302875152512), (9.952538678825942, 0.0765912655059405), (10.633116883116884, 0.3868063643113476), (8.962623951182303, 0.5356543501521245), (24.424052028218693, 0.0001799341000410295), (33.458333333333336, 3.0515495359883186e-06), (7.5900727428326915, 0.1803216251525998), (1.2193700931405211, 0.9430056230417913), (4.13396127681842, 0.5302945922960001), (13.621744994294014, 0.018199348700816576), (9.95953425229741, 0.4440506480627435), (0.8163784796087202, 0.9759769308325452), (0.8163784796087202, 0.9759769308325452), (7.706256193409676, 0.6575045784047995), (0.8604564160119714, 0.9730180743943798), (1.7048794826572602, 0.8882844551788736)]
